In [1]:
! pip install datasets -q

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

In [3]:
dataset=load_dataset(
    "conll2003",
    trust_remote_code=True
).remove_columns(["id"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset["train"].to_pandas().head()

,tokens,pos_tags,chunk_tags,ner_tags
0,"[EU, rejects, German, call, to, boycott, Briti...","[22, 42, 16, 21, 35, 37, 16, 21, 7]","[11, 21, 11, 12, 21, 22, 11, 12, 0]","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,"[Peter, Blackburn]","[22, 22]","[11, 12]","[1, 2]"
2,"[BRUSSELS, 1996-08-22]","[22, 11]","[11, 12]","[5, 0]"
3,"[The, European, Commission, said, on, Thursday...","[12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 3...","[11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 1...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ..."
4,"[Germany, 's, representative, to, the, Europea...","[22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 2...","[11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 1...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ..."


In [5]:
#"B-":beginning "I-":inside --> start of the phrase , part of previous phrase
label2id={
    "O": 0, "B-PER": 1, "I-PER": 2, "B-ORG": 3, "I-ORG": 4,"B-LOC": 5, "I-LOC": 6, "B-MISC": 7, "I-MISC": 8
}
id2label={v:k for k,v in label2id.items()}
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [6]:
example=dataset["train"][848]
example

{'tokens': ['Dean',
  'Palmer',
  'hit',
  'his',
  '30th',
  'homer',
  'for',
  'the',
  'Rangers',
  '.'],
 'pos_tags': [22, 22, 38, 29, 16, 21, 15, 12, 23, 7],
 'chunk_tags': [11, 12, 21, 11, 12, 12, 13, 11, 12, 0],
 'ner_tags': [1, 2, 0, 0, 0, 0, 0, 0, 3, 0]}

In [7]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")
#model=AutoModel.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
token_id=tokenizer.encode(example["tokens"],is_split_into_words=True)
token_id

[101, 4285, 8450, 1855, 1117, 13631, 1313, 1197, 1111, 1103, 6838, 119, 102]

In [9]:
tokenizer.encode(" ".join(example["tokens"])) # is_split_into_words

[101, 4285, 8450, 1855, 1117, 13631, 1313, 1197, 1111, 1103, 6838, 119, 102]

In [10]:
tokenizer.convert_ids_to_tokens(token_id)

['[CLS]',
 'Dean',
 'Palmer',
 'hit',
 'his',
 '30th',
 'home',
 '##r',
 'for',
 'the',
 'Rangers',
 '.',
 '[SEP]']

In [11]:
# the root of each token
token_ids=tokenizer(
      example["tokens"],
      is_split_into_words=True,
      truncation=True
  )
token_ids.word_ids()

[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, None]

In [12]:
def align_labels(examples):
  token_ids=tokenizer(
      examples["tokens"],
      truncation=True,
      is_split_into_words=True
  )
  labels=examples["ner_tags"]
  updated_labels=[]
  for idx,label in enumerate(labels):
    word_ids=token_ids.word_ids(batch_index=idx)
    previous_word_idx=None
    label_ids=[]
    for word_idx in word_ids:
      if word_idx!=previous_word_idx:
        previous_word_idx=word_idx
        #print(label)
        updated_label= -100 if word_idx is None else label[word_idx]
        label_ids.append(updated_label)
      elif word_idx is None:
        label_ids.append(-100)
      else:
        updated_label=label[word_idx]
        if updated_label%2==1:
          updated_label+=1
        label_ids.append(updated_label)
    updated_labels.append(label_ids)
  token_ids["labels"]=updated_labels
  return token_ids

In [13]:
tokenized=dataset.map(align_labels,batched=True)

In [14]:
tokenized["train"][848]

{'tokens': ['Dean',
  'Palmer',
  'hit',
  'his',
  '30th',
  'homer',
  'for',
  'the',
  'Rangers',
  '.'],
 'pos_tags': [22, 22, 38, 29, 16, 21, 15, 12, 23, 7],
 'chunk_tags': [11, 12, 21, 11, 12, 12, 13, 11, 12, 0],
 'ner_tags': [1, 2, 0, 0, 0, 0, 0, 0, 3, 0],
 'input_ids': [101,
  4285,
  8450,
  1855,
  1117,
  13631,
  1313,
  1197,
  1111,
  1103,
  6838,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, -100]}

In [15]:
len(token_id)==len(tokenized["train"][848]["labels"])

True

# Evaluation

In [16]:
! pip install evaluate seqeval -q

In [17]:
import evaluate
import numpy as np

seqeval=evaluate.load("seqeval")

In [18]:
def compute_metric(eval_pred):
  # eval_pred (logits,labels)
  logits,labels=eval_pred
  # logits shape (b,n_tokens,n_labels), labels_shape (b,n_token)
  predictions=np.argmax(logits,dim=2) # predictions shape (b,n_tokens,1)

  true_pres=[]
  true_labels=[]
  for pred,label in zip(predictions,labels):
    for token_pred,token_label in zip(pred,label):
      if token_label!=-100:
        true_pres.append([id2label[token_pred]])
        true_labels.append([id2label[token_label]])
  results=seqeval.compute(
      predictions=true_pres,references=true_labels
  )
  return {"f1":results["overall_f1"]}


# Fine Tuning Bert

In [19]:
! nvidia-smi

Tue Oct 29 13:50:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
from transformers.trainer import TrainingArguments,Trainer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [21]:
model=AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
    )
model=model.to("cuda")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import torch
model(torch.tensor([tokenizer.encode("Hello From me!")]).to("cuda"))

TokenClassifierOutput(loss=None, logits=tensor([[[-0.3278, -0.0306,  0.1691,  0.2251, -0.1014, -0.7512,  0.1705,
          -0.2649, -0.3330],
         [-0.3787,  0.3154,  0.0583,  0.1973, -0.2377, -0.7580,  0.1071,
          -0.3590, -0.5915],
         [ 0.0057,  0.0916, -0.0197, -0.1194, -0.2122, -0.4089,  0.1568,
          -0.1600, -0.2075],
         [ 0.0369, -0.0212, -0.0729, -0.0329, -0.2873, -0.0314, -0.1700,
          -0.2954, -0.3013],
         [ 0.1317, -0.0447,  0.2847, -0.4128, -0.3720,  0.0343, -0.1409,
          -0.2729, -0.2554],
         [ 0.0396,  0.3617, -0.6330, -0.4231,  0.3907,  0.0203,  0.2813,
          -0.2573, -0.2564]]], device='cuda:0', grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [23]:
# ft model "bert-base-cased"
args=TrainingArguments(
    "nerclassifications",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

In [24]:
datacollator=DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=datacollator,
    compute_metrics=compute_metric
)

In [26]:
trainer.train()

Step,Training Loss
500,0.222700


TrainOutput(global_step=878, training_loss=0.16162384180924888, metrics={'train_runtime': 167.6079, 'train_samples_per_second': 83.773, 'train_steps_per_second': 5.238, 'total_flos': 351240792638148.0, 'train_loss': 0.16162384180924888, 'epoch': 1.0})

In [29]:
trainer.save_model("ner_model")

In [33]:
from transformers import pipeline
token_classifier=pipeline(
    "token-classification",
    model="ner_model"
)
token_classifier("my name Abdeljalil")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-PER',
  'score': 0.98813665,
  'index': 3,
  'word': 'Abd',
  'start': 8,
  'end': 11},
 {'entity': 'I-PER',
  'score': 0.98896563,
  'index': 4,
  'word': '##el',
  'start': 11,
  'end': 13},
 {'entity': 'I-PER',
  'score': 0.9941147,
  'index': 5,
  'word': '##ja',
  'start': 13,
  'end': 15},
 {'entity': 'I-PER',
  'score': 0.99390644,
  'index': 6,
  'word': '##li',
  'start': 15,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.98908395,
  'index': 7,
  'word': '##l',
  'start': 17,
  'end': 18}]

In [27]:
#import gc
gc.collect()
torch.cuda.empty_cache()

NameError: name 'gc' is not defined

In [ ]:
TrainingArguments?